Version 1.0.1

# Check your versions

In [1]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm 

for p in [np, pd, scipy, sklearn, lightgbm]:
    print (p.__name__, p.__version__)

numpy 1.19.1
pandas 1.1.1
scipy 1.5.2
sklearn 0.23.2
lightgbm 3.0.0


**Important!** There is a huge chance that the assignment will be impossible to pass if the versions of `lighgbm` and `scikit-learn` are wrong. The versions being tested:

    numpy 1.13.1
    pandas 0.20.3
    scipy 0.19.1
    sklearn 0.19.0
    ligthgbm 2.0.6
    

To install an older version of `lighgbm` you may use the following command:
```
pip uninstall lightgbm
pip install lightgbm==2.0.6
```

# Ensembling

In this programming assignment you are asked to implement two ensembling schemes: simple linear mix and stacking.

We will spend several cells to load data and create feature matrix, you can scroll down this part or try to understand what's happening.

In [2]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_rows', 20)
# pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
# from tqdm import tqdm_notebook
from tqdm.notebook import tqdm

from itertools import product


def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

# Load data subset

Let's load the data from the hard drive first.

In [3]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')
shops = pd.read_csv('../readonly/final_project_data/shops.csv')
items = pd.read_csv('../readonly/final_project_data/items.csv')
item_cats = pd.read_csv('../readonly/final_project_data/item_categories.csv')
tests = pd.read_csv('../readonly/final_project_data/test.csv.gz')

In [4]:
np.sort(
    tests['shop_id'].unique())

array([ 2,  3,  4,  5,  6,  7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25,
       26, 28, 31, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49,
       50, 52, 53, 55, 56, 57, 58, 59])

In [5]:
## And use only 3 shops for simplicity.
# sales = sales[sales['shop_id'].isin([26, 27, 28])]
# tests = tests[tests['shop_id'].isin([26, 27, 28])]

In [6]:
# Remove outliers
sales = sales[sales['item_price']<100000]
sales = sales[sales['item_cnt_day']<1001]

# Get a feature matrix

We now need to prepare the features. This part is all implemented for you.

In [7]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
grid

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0
...,...,...,...
10913799,21,7635,33
10913800,21,7638,33
10913801,21,7640,33
10913802,21,7632,33


In [8]:
## Append test date to our grid
tests['date_block_num'] = 34
grid = grid.append(tests.drop(columns=['ID']), ignore_index=True)
grid

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0
...,...,...,...
11127999,45,18454,34
11128000,45,16188,34
11128001,45,15757,34
11128002,45,19648,34


In [9]:
# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'})
gb.rename(columns={'item_cnt_day':'target'}, inplace=True)

# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)
all_data

,shop_id,item_id,date_block_num,target
0,59,22154,0,1.0
1,59,2552,0,0.0
2,59,2554,0,0.0
3,59,2555,0,0.0
4,59,2564,0,0.0
...,...,...,...,...
11127999,45,18454,34,0.0
11128000,45,16188,34,0.0
11128001,45,15757,34,0.0
11128002,45,19648,34,0.0


In [10]:
# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})
gb.rename(columns={'item_cnt_day':'target_shop'}, inplace=True)
                                                                      
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)
all_data

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})
gb.rename(columns={'item_cnt_day':'target_item'}, inplace=True)

all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
del grid, gb 
gc.collect();

In [11]:
print('Shape of "all_data":', all_data.shape[0], 'rows,', all_data.shape[1], 'columns')
all_data

Shape of "all_data": 11128004 rows, 6 columns


,shop_id,item_id,date_block_num,target,target_shop,target_item
0,59,22154,0,1.0,2017.0,18.0
1,59,2552,0,0.0,2017.0,0.0
2,59,2554,0,0.0,2017.0,1.0
3,59,2555,0,0.0,2017.0,2.0
4,59,2564,0,0.0,2017.0,5.0
...,...,...,...,...,...,...
11127999,45,18454,34,0.0,0.0,0.0
11128000,45,16188,34,0.0,0.0,0.0
11128001,45,15757,34,0.0,0.0,0.0
11128002,45,19648,34,0.0,0.0,0.0


In [12]:
all_data.to_parquet('all_data_first.parquet', index=True)

In [13]:
all_data = pd.read_parquet('all_data_first.parquet')

After creating a grid, we can calculate some features. We will use lags from [1, 2, 3, 4, 5, 12] months ago.

In [14]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

for month_shift in tqdm(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
print('fit_cols =', fit_cols)
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 
print('to_drop_cols =', to_drop_cols)
# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();


to_drop_cols = ['target_item', 'target', 'target_shop', 'date_block_num']


To this end, we've created a feature matrix. It is stored in `all_data` variable. Take a look:

In [15]:
# all_data.head(5)
all_data.to_parquet('all_data.parquet', index=True)
all_data

,shop_id,item_id,date_block_num,target,target_shop,target_item,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,target_lag_3,target_item_lag_3,target_shop_lag_3,target_lag_4,target_item_lag_4,target_shop_lag_4,target_lag_5,target_item_lag_5,target_shop_lag_5,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id
0,54,10297,12,4.0,8198.0,23.0,3.0,42.0,10055.0,0.0,2.0,7978.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
1,54,10296,12,3.0,8198.0,17.0,0.0,24.0,10055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38
2,54,10298,12,14.0,8198.0,182.0,21.0,369.0,10055.0,119.0,1309.0,7978.0,7.0,144.0,6676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
3,54,10300,12,3.0,8198.0,26.0,1.0,54.0,10055.0,31.0,361.0,7978.0,0.0,53.0,6676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
4,54,10284,12,1.0,8198.0,3.0,0.0,4.0,10055.0,0.0,3.0,7978.0,0.0,5.0,6676.0,0.0,3.0,7827.0,0.0,10.0,7792.0,0.0,0.0,0.0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639289,45,18454,34,0.0,0.0,0.0,1.0,2.0,702.0,0.0,1.0,654.0,0.0,3.0,710.0,0.0,12.0,675.0,0.0,19.0,622.0,0.0,0.0,0.0,55
6639290,45,16188,34,0.0,0.0,0.0,0.0,1.0,702.0,0.0,3.0,654.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64
6639291,45,15757,34,0.0,0.0,0.0,0.0,5.0,702.0,0.0,3.0,654.0,0.0,4.0,710.0,0.0,4.0,675.0,0.0,8.0,622.0,0.0,9.0,1251.0,55
6639292,45,19648,34,0.0,0.0,0.0,0.0,2.0,702.0,0.0,3.0,654.0,0.0,7.0,710.0,0.0,2.0,675.0,0.0,4.0,622.0,0.0,0.0,0.0,40


In [16]:
# all_data[all_data['date_block_num']==34]

# Train/test split

For a sake of the programming assignment, let's artificially split the data into train and test. We will treat last month data as the test set.

In [17]:
all_data = pd.read_parquet('all_data.parquet')

In [18]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = dates.max()-1
print('Test `date_block_num` is %d' % last_block)

Test `date_block_num` is 33


In [19]:
dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

In [20]:
X_train

,shop_id,item_id,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,target_lag_3,target_item_lag_3,target_shop_lag_3,target_lag_4,target_item_lag_4,target_shop_lag_4,target_lag_5,target_item_lag_5,target_shop_lag_5,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id
0,54,10297,3.0,42.0,10055.0,0.0,2.0,7978.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
1,54,10296,0.0,24.0,10055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38
2,54,10298,21.0,369.0,10055.0,119.0,1309.0,7978.0,7.0,144.0,6676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
3,54,10300,1.0,54.0,10055.0,31.0,361.0,7978.0,0.0,53.0,6676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
4,54,10284,0.0,4.0,10055.0,0.0,3.0,7978.0,0.0,5.0,6676.0,0.0,3.0,7827.0,0.0,10.0,7792.0,0.0,0.0,0.0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6186917,27,21279,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3357.0,1.0,1.0,3786.0,0.0,0.0,0.0,0.0,0.0,0.0,61
6186918,27,21283,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3357.0,0.0,0.0,0.0,0.0,1.0,3518.0,0.0,4.0,4026.0,61
6186919,27,21352,0.0,0.0,0.0,1.0,2.0,2478.0,0.0,0.0,0.0,1.0,1.0,3786.0,0.0,0.0,0.0,0.0,2.0,4026.0,37
6186920,27,21284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3357.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61


# First level models 

You need to implement a basic stacking scheme. We have a time component here, so we will use ***scheme f)*** from the reading material. Recall, that we always use first level models to build two datasets: test meta-features and 2-nd level train-metafetures. Let's see how we get test meta-features first. 

### Test meta-features

Firts, we will run *linear regression* on numeric columns and get predictions for the last month.

In [21]:
from sklearn.metrics import mean_squared_error

In [48]:
model_lr = LinearRegression()
model_lr.fit(X_train.values, y_train)
pred_lr = model_lr.predict(X_test.values)

print('Test R-squared for linreg is %f' % r2_score(y_test, pred_lr))
print('Test RMSE linreg is %f' % np.sqrt(mean_squared_error(y_test, pred_lr)))

Test R-squared for linreg is 0.481407
Test RMSE linreg is 1.941356


And the we run *LightGBM*.

In [50]:
%%time
# LightGGM
model_lgb = lgb.LGBMRegressor(
    n_estimators=1000,
    n_jobs=-1,
    objective='mse',
    random_state=17
)
model_lgb.fit(X_train, y_train, 
              eval_set=(X_test, y_test), 
              early_stopping_rounds=100,
              verbose=100)
pred_lgb = model_lgb.predict(X_test)

print('Test R-squared for LightGBM is %f' % r2_score(y_test, pred_lgb))
print('Test RMSE LightGBM is %f' % np.sqrt(mean_squared_error(y_test, pred_lgb)))

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4.70444
Early stopping, best iteration is:
[14]	valid_0's l2: 4.07082
Test R-squared for LightGBM is 0.453734
Test RMSE LightGBM is 1.992481
CPU times: user 1min 37s, sys: 3.87 s, total: 1min 41s
Wall time: 37.3 s


In [49]:
%%time
# Catboost
import catboost as cb

cat_features = ['shop_id', 'item_id', 'item_category_id']
cb_train = cb.Pool(X_train, label=y_train, cat_features=cat_features)
cb_test = cb.Pool(X_test, label=y_test, cat_features=cat_features)

model_cbr = cb.CatBoostRegressor(loss_function='RMSE')
model_cbr.fit(cb_train, 
              eval_set=cb_test, 
              verbose=100, 
              early_stopping_rounds=100)
pred_cb = model_cbr.predict(X_test)

print('Test R-squared for CatBoost is %f' % r2_score(y_test, pred_cb))
print('Test RMSE CatBoost is %f' % np.sqrt(mean_squared_error(y_test, pred_cb)))

Learning rate set to 0.239925
0:	learn: 3.1898125	test: 2.4052391	best: 2.4052391 (0)	total: 2.53s	remaining: 42m 10s
100:	learn: 1.7579511	test: 2.3472933	best: 1.9260270 (8)	total: 4m 2s	remaining: 35m 58s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 1.92602704
bestIteration = 8

Shrink model to first 9 iterations.
Test R-squared for CatBoost is 0.489565
Test RMSE CatBoost is 1.926027
CPU times: user 14min 13s, sys: 50 s, total: 15min 3s
Wall time: 4min 31s


In [56]:
%%time
# XGBM
import xgboost as xgb

model_xgb = xgb.XGBRegressor(n_estimators=500, n_jobs=4, random_state=17)
model_xgb.fit(X_train, y_train, 
              eval_set=[(X_test, y_test)], 
              eval_metric="rmse", 
              verbose=10, 
              early_stopping_rounds = 30)
pred_xgb = model_xgb.predict(X_test)

print('Test R-squared for XGBoost is %f' % r2_score(y_test, pred_xgb))
print('Test RMSE XGBoost is %f' % np.sqrt(mean_squared_error(y_test, pred_xgb)))

[12:34:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:2.36626
Will train until validation_0-rmse hasn't improved in 30 rounds.
[10]	validation_0-rmse:2.06379
[20]	validation_0-rmse:2.06608
[30]	validation_0-rmse:2.14833
Stopping. Best iteration:
[5]	validation_0-rmse:2.04074

Test R-squared for XGBoost is 0.427007
Test RMSE XGBoost is 2.040642
CPU times: user 15min 1s, sys: 11.8 s, total: 15min 13s
Wall time: 4min 19s


Finally, concatenate test predictions to get test meta-features.

In [61]:
X_test_level2 = np.c_[pred_lr, pred_lgb, pred_cb, pred_xgb] 
X_test_level2.tofile('X_test_level2')

In [58]:
X_test_level2, X_test_level2.shape

(array([[0.12520302, 0.05967719, 0.223819  , 0.1877704 ],
        [0.77716301, 0.84772476, 0.64272452, 0.81712544],
        [0.99906431, 0.51660697, 0.3269396 , 0.51259017],
        ...,
        [0.10154384, 0.22052162, 0.223819  , 0.22039822],
        [0.12411467, 0.06343504, 0.223819  , 0.18701917],
        [0.05064061, 0.04475217, 0.1307609 , 0.10197899]]),
 (238172, 4))

### Train meta-features

**Now it is your turn to write the code**. You need to implement ***scheme f)*** from the reading material. Here, we will use duration **T** equal to month and **M=15**.  

That is, you need to get predictions (meta-features) from *linear regression* and *LightGBM* for months 27, 28, 29, 30, 31, 32. Use the same parameters as in above models.

In [35]:
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

In [42]:
# X_train_level2, X_train_level2.shape, y_train_level2, y_train_level2.shape
y_train_level2.shape

(1376192,)

In [ ]:
%%time
# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 4])

M=15

X_list_2 = []

# Now fill `X_train_level2` with metafeatures
for cur_block_num in tqdm([27, 28, 29, 30, 31, 32]):
    
    print('Current block number:', cur_block_num)
    
    '''
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit LightGBM and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    '''      
    
    #  YOUR CODE GOES HERE
    cdates = (dates <  cur_block_num)
    cur_dates = (dates==cur_block_num)
    cX_test = all_data.loc[cdates].drop(to_drop_cols, axis=1)
    cy_train = all_data.loc[cdates, 'target'].values
    cX_pred = all_data.loc[cur_dates].drop(to_drop_cols, axis=1)
    cy_pred = all_data.loc[cur_dates, 'target'].values
    
    # LinearRegression
    lr_c = LinearRegression()
    lr_c.fit(cX_test.values, cy_train)
    pred_lr_c = lr_c.predict(cX_pred)
    print('Train RMSE for LinReg is %f' % np.sqrt(mean_squared_error(cy_pred, pred_lr_c)))
    
    # LGMRegression
    model_lgb_c = lgb.LGBMRegressor(n_estimators=1000,
                                    n_jobs=-1,
                                    objective='mse',
                                    random_state=17)
    model_lgb_c.fit(cX_test, cy_train, 
              eval_set=(cX_pred, cy_pred), 
              early_stopping_rounds=100,
              verbose=100)
    pred_lgb_c = model_lgb_c.predict(cX_pred)
    print('Train RMSE for LightGBM is %f' % np.sqrt(mean_squared_error(cy_pred, pred_lgb_c)))
    #model_lgb_c = lgb.train(lgb_params, lgb.Dataset(cX_test, label=cy_train), 100)
    #pred_lgb_c = model_lgb_c.predict(cX_pred)
    
    # Catboost Regression
    model_cbr_c = cb.CatBoostRegressor(loss_function='RMSE')
    model_cbr_c.fit(cb.Pool(cX_test, label=cy_train, cat_features=cat_features), 
                  eval_set=cb.Pool(cX_pred, label=cy_pred, cat_features=cat_features), 
                  verbose=100, 
                  early_stopping_rounds=100)
    pred_cbr_c = model_cbr_c.predict(cX_pred)
    print('Train RMSE for Catboost is %f' % np.sqrt(mean_squared_error(cy_pred, pred_cbr_c)))
    
    # XGBM
    model_xgb_c = xgb.XGBRegressor(n_estimators=500, n_jobs=4, random_state=17)
    model_xgb_c.fit(cX_test, cy_train, 
                  eval_set=[(cX_pred, cy_pred)], 
                  eval_metric="rmse", 
                  verbose=20, 
                  early_stopping_rounds = 30)
    pred_xgb_c = model_xgb_c.predict(cX_pred)
    print('Train RMSE for XGBM is %f' % np.sqrt(mean_squared_error(cy_pred, pred_xgb_c)))
    
    X_list_2.append(np.c_[pred_lr_c, pred_lgb_c, pred_cbr_c, pred_xgb_c])

X_train_level2 = np.vstack(X_list_2)
# print(X_train_level2.mean(axis=0))
# Sanity check
# assert np.all(np.isclose(X_train_level2.mean(axis=0), [ 1.50148988,  1.38811989]))

Current block number: 27
Train RMSE for LinReg is 4.213454
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 19.6814
Early stopping, best iteration is:
[36]	valid_0's l2: 19.2313
Train RMSE for LightGBM is 4.385353
Learning rate set to 0.228784
0:	learn: 2.9925644	test: 4.5316163	best: 4.5316163 (0)	total: 1.84s	remaining: 30m 41s
100:	learn: 1.6146711	test: 4.4430876	best: 4.3720309 (18)	total: 3m 8s	remaining: 27m 54s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 4.372030867
bestIteration = 18

Shrink model to first 19 iterations.
Train RMSE for Catboost is 4.372031
[13:01:19] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1598185621802/work/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:4.49834
Will train until validation_0-rmse hasn't improved in 30 rounds.
[

In [ ]:
X_train_level2, X_train_level2.shape

Remember, the ensembles work best, when first level models are diverse. We can qualitatively analyze the diversity by examinig *scatter plot* between the two metafeatures. Plot the scatter plot below. values

In [ ]:
# YOUR CODE GOES HERE
plt.scatter(X_train_level2[:,0], X_train_level2[:,1])

# Ensembling

Now, when the meta-features are created, we can ensemble our first level models.

### Simple convex mix

Let's start with simple linear convex mix:

$$
mix= \alpha\cdot\text{linreg_prediction}+(1-\alpha)\cdot\text{lgb_prediction}
$$

We need to find an optimal $\alpha$. And it is very easy, as it is feasible to do grid search. Next, find the optimal $\alpha$ out of `alphas_to_try` array. Remember, that you need to use train meta-features (not test) when searching for $\alpha$. 

In [ ]:
alphas_to_try = np.linspace(0, 1, 1001)

best_score = 0
# best_alpha = 0

for alpha in alphas_to_try:
    mix = alpha*X_train_level2[:,0] + (1-alpha)*X_train_level2[:,1]
    #np.c_[X_train_level2, mix]
    score = r2_score(y_train_level2, mix)
    # print(score)
    if score > best_score:
        best_score = score
        best_alpha = alpha
        best_mix = mix

# YOUR CODE GOES HERE
best_alpha = best_alpha # YOUR CODE GOES HERE
r2_train_simple_mix = best_score # YOUR CODE GOES HERE

print('Best alpha: %f; Corresponding r2 score on train: %f' % (best_alpha, r2_train_simple_mix))

Now use the $\alpha$ you've found to compute predictions for the test set 

In [ ]:
test_preds = best_alpha*X_test_level2[:,0] + (1-best_alpha)*X_test_level2[:,1] # YOUR CODE GOES HERE
r2_test_simple_mix = r2_score(y_test, test_preds) # YOUR CODE GOES HERE

print('Test R-squared for simple mix is %f' % r2_test_simple_mix)

### Stacking

Now, we will try a more advanced ensembling technique. Fit a linear regression model to the meta-features. Use the same parameters as in the model above.

In [ ]:
# YOUR CODE GOES HERE
lr_meta = LinearRegression()
# X_train_l2_meta = np.c_[X_train_level2, best_mix]
# X_test_l2_meta = np.c_[X_test_level2, test_preds]

lr_meta.fit(X_train_level2, y_train_level2)
# pred_lr_meta = lr_meta.predict(X_test_l2_meta)
# print('Test R-squared for linreg is %f' % r2_score(y_test, pred_lr))

Compute R-squared on the train and test sets.

In [ ]:
train_preds = lr_meta.predict(X_train_level2) # YOUR CODE GOES HERE
r2_train_stacking = r2_score(y_train_level2, train_preds) # YOUR CODE GOES HERE

test_preds = lr_meta.predict(X_test_level2) # YOUR CODE GOES HERE
r2_test_stacking = r2_score(y_test, test_preds) # YOUR CODE GOES HERE

print('Train R-squared for stacking is %f' % r2_train_stacking)
print('Test  R-squared for stacking is %f' % r2_test_stacking)

Interesting, that the score turned out to be lower than in previous method. Although the model is very simple (just 3 parameters) and, in fact, mixes predictions linearly, it looks like it managed to overfit. **Examine and compare** train and test scores for the two methods. 

And of course this particular case does not mean simple mix is always better than stacking.

We all done! Submit everything we need to the grader now.

In [ ]:
from grader import Grader
grader = Grader()

grader.submit_tag('best_alpha', best_alpha)

grader.submit_tag('r2_train_simple_mix', r2_train_simple_mix)
grader.submit_tag('r2_test_simple_mix',  r2_test_simple_mix)

grader.submit_tag('r2_train_stacking', r2_train_stacking)
grader.submit_tag('r2_test_stacking',  r2_test_stacking)

In [ ]:
STUDENT_EMAIL = 'ddementiev@yandex.ru' # EMAIL HERE
STUDENT_TOKEN = '9o3OQup3ZKX2Ryho' # TOKEN HERE
grader.status()

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)